In [ ]:
import warnings
warnings.filterwarnings('ignore')

# import sys
# sys.path.insert(1, '../') # make sure to use the code in this repo
# 
# import os
# os.environ['CDF_LIB'] = '/home/peijin/cdf/cdf38_0-dist/lib'

import glob
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from spacepy import pycdf
from astropy.io import fits as pyfits
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib as mpl
# try to use the precise epoch 
mpl.rcParams['date.epoch'] = '1970-01-01T00:00:00'
try:
    mdates.set_epoch('1970-01-01T00:00:00')
except:
    pass
data_dir = '/home/mnedal/data'

In [ ]:
date = '2021-09-18'
year, month, day = date.split('-')

nda_files = sorted(glob.glob(f'{data_dir}/NDA/NDA_{day}{month}{year}_*.fits'))
nda_files

In [ ]:
nda_data = pyfits.open(nda_files[0])
nda_data.info()

In [ ]:
nda_data[0].header

In [ ]:
nda_data[1].header

In [ ]:
nda_data[2].header

In [ ]:
nda_data[3].header

In [ ]:
nda_data[4].header

In [ ]:
### define x and y axes
nda_freq = pd.DataFrame(nda_data[3].data).values.reshape(-1)
nda_time = pd.DataFrame(nda_data[4].data).values.reshape(-1)
nda_arr1 = nda_data[1].data[:, :-1]
nda_arr2 = nda_data[2].data[:, :-1]

datetime_obj_start = nda_data[0].header['DATE-OBS'] + ' ' + nda_data[0].header['TIME-OBS']
datetime_obj_end   = nda_data[0].header['DATE-OBS'] + ' ' + nda_data[0].header['TIME-END']

start_obs_time = datetime.strptime(datetime_obj_start, '%d/%m/%Y %H:%M:%S')
end_obs_time   = datetime.strptime(datetime_obj_end, '%d/%m/%Y %H:%M:%S')

# Convert time data to datetime objects
time_data = nda_data[4].data
datetimes_list = [pd.Timestamp('2021-09-18 00:00:00') + timedelta(seconds=float(time[0])) for time in time_data]

In [ ]:
datetimes_list[0], datetimes_list[-1]

In [ ]:
datetime_obj_start, datetime_obj_end

In [ ]:
len(datetimes_list)

In [ ]:
nda_arr1.shape

In [ ]:
fig = plt.figure(figsize=[8,7])

ax = fig.add_subplot(211)
im1 = ax.imshow(nda_arr1, 
           aspect='auto', origin='lower',
        #    vmin=(np.mean(nda_arr1)-2 * np.std(nda_arr1)),
        #    vmax=(np.mean(nda_arr1)+3 * np.std(nda_arr1)),
    vmin=np.percentile(nda_arr1, 1),
    vmax=np.percentile(nda_arr1, 95),
           extent=[mdates.date2num(datetimes_list[0]), mdates.date2num(datetimes_list[-1]), nda_freq[0], nda_freq[-1]],
           cmap='inferno')

ax.set_ylabel('Frequency (MHz)')
ax.set_title(nda_data[0].header['ORIGIN'] + ' [1] ' + nda_data[0].header['DATE-OBS'])
ax.xaxis_date()
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
plt.colorbar(im1, label='im1')

ax = fig.add_subplot(212)
im2 = ax.imshow(nda_arr2,
           aspect='auto', origin='lower',
        #    vmin=(np.mean(nda_arr2)-2 * np.std(nda_arr2)),
        #    vmax=(np.mean(nda_arr2)+3 * np.std(nda_arr2)),
    vmin=np.percentile(nda_arr2, 1),
    vmax=np.percentile(nda_arr2, 95),
           extent=[mdates.date2num(datetimes_list[0]), mdates.date2num(datetimes_list[-1]), nda_freq[0], nda_freq[-1]],
           cmap='inferno')

ax.set_xlabel('Time (UT)')
ax.set_ylabel('Frequency (MHz)')
ax.set_title(nda_data[0].header['ORIGIN'] + ' [2] ' + nda_data[0].header['DATE-OBS'])
ax.xaxis_date()
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
plt.colorbar(im2, label='im2')

fig.tight_layout()
plt.show()

In [ ]:
nda_diff = nda_arr2 - nda_arr1

fig = plt.figure(figsize=[8,7])

ax = fig.add_subplot(211)
im = ax.imshow(nda_diff, 
           aspect='auto', origin='lower',
    vmin=np.percentile(nda_diff, 1),
    vmax=np.percentile(nda_diff, 70),
           extent=[mdates.date2num(datetimes_list[0]), mdates.date2num(datetimes_list[-1]), nda_freq[0], nda_freq[-1]],
           cmap='inferno')

ax.set_xlabel('Time (UT)')
ax.set_ylabel('Frequency (MHz)')
ax.set_title('Difference ' + nda_data[0].header['DATE-OBS'])
ax.xaxis_date()
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
plt.colorbar(im2, label='im')

fig.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=[8,7])

ax = fig.add_subplot(211)
im1 = ax.pcolormesh(datetimes_list, nda_freq, nda_arr1,
    vmin=np.percentile(nda_arr1, 1),
    vmax=np.percentile(nda_arr1, 95),
           cmap='inferno')

ax.set_ylabel('Frequency (MHz)')
ax.set_title(nda_data[0].header['ORIGIN'] + ' [1] ' + nda_data[0].header['DATE-OBS'])
ax.xaxis_date()
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
plt.colorbar(im1, label='im1')

ax = fig.add_subplot(212)
im2 = ax.pcolormesh(datetimes_list, nda_freq, nda_arr2,
    vmin=np.percentile(nda_arr2, 1),
    vmax=np.percentile(nda_arr2, 95),
           cmap='inferno')

ax.set_xlabel('Time (UT)')
ax.set_ylabel('Frequency (MHz)')
ax.set_title(nda_data[0].header['ORIGIN'] + ' [2] ' + nda_data[0].header['DATE-OBS'])
ax.xaxis_date()
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
plt.colorbar(im2, label='im2')

fig.tight_layout()
plt.show()

In [ ]:
fig = plt.figure(figsize=[8,4])
ax = fig.add_subplot(111)
im1 = ax.pcolormesh(datetimes_list, nda_freq, nda_arr1,
                    vmin=np.percentile(nda_arr1, 1),
                    vmax=np.percentile(nda_arr1, 95),
                    cmap='inferno')

ax.set_xlim(left=pd.Timestamp('2021-09-18 08:00'), right=pd.Timestamp('2021-09-18 09:00'))
ax.set_xlabel('Time (UT)')
ax.set_ylabel('Frequency (MHz)')
ax.set_title(nda_data[0].header['ORIGIN'] + ' [1] ' + nda_data[0].header['DATE-OBS'])
ax.xaxis_date()
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
plt.colorbar(im1, pad=0.02, label='im1')
fig.tight_layout()
plt.show()

In [ ]:
nda_arr1.shape, nda_arr2.shape, len(datetimes_list), len(time_data)

In [ ]:
fig = plt.figure(figsize=[10,5])

ax = fig.add_subplot(211)
ax.plot(datetimes_list, nda_arr1[70,:], label='arr 1')
ax.plot(datetimes_list, nda_arr2[70,:], label='arr 2')
ax.set_ylabel('Frequency (MHz)')
ax.legend()
ax.xaxis_date()
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

ax = fig.add_subplot(212)
ax.plot(datetimes_list, nda_diff[70,:], label='Diff')
ax.set_xlabel('Time (UT)')
ax.set_ylabel('Frequency (MHz)')
ax.legend()
ax.xaxis_date()
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))

fig.tight_layout()
plt.show()